In [2]:
#Detection fraude - Feature engineering (l'objectif est d'extraire les caracteristiques pertinentes a partir des donnees netoyees, en vue de detecter les fraudes liee aux transactions)

#1) importation des bibliotheques
import pandas as pd #lire et manipuler lesfichiers cvs
import numpy as np #calcul numerique
import os #chemin au datasets
from datetime import datetime, timedelta 

#2) chargement des datasets nettoyes
#definition du chemin depuis le notebook vers le dossier contenant les datasets
data_path = os.path.join("..", "datasets") #".." pour remonter au dossier parent

# chargement des fichiers nettoyees
account_df = pd.read_csv(os.path.join(data_path, "account_cleaned.csv"))
user_df = pd.read_csv(os.path.join(data_path, "user_cleaned.csv"))
history_df = pd.read_csv(os.path.join(data_path, "history_cleaned.csv"))
bill_df = pd.read_csv(os.path.join(data_path, "bill_cleaned.csv"))
transfert_df = pd.read_csv(os.path.join(data_path, "transfert_cleaned.csv"))
splogin_history_df = pd.read_csv(os.path.join(data_path, "splogin_cleaned.csv"))
changes_df = pd.read_csv(os.path.join(data_path, "changes_cleaned.csv"))
proceed_df = pd.read_csv(os.path.join(data_path, "proceed_cleaned.csv"))

print("\n Tous les fichiers CSV nettoyes ont été chargés avec succès !")


 Tous les fichiers CSV nettoyes ont été chargés avec succès !


In [3]:
#3) extraction des caracteristiques par classe

#a) classe history

# S'assurer que la colonne 'date' est bien de type datetime
history_df['date'] = pd.to_datetime(history_df['date'])

# Créer un nouveau DataFrame pour stocker uniquement les features extraites
history_features = pd.DataFrame()

# Clé de jointure pour la fusion future
history_features['user_id'] = history_df['user_id']

# 1. Heure de la transaction (0 à 23)
history_features['hour'] = history_df['date'].dt.hour

# 2. Jour de la semaine (0 = lundi, 6 = dimanche)
history_features['day_of_week'] = history_df['date'].dt.dayofweek

# 3. Indicateur si la transaction a eu lieu un week-end (1 = oui, 0 = non)
history_features['is_weekend'] = history_features['day_of_week'].isin([5, 6]).astype(int)

# 4. Variation de solde autour de la transaction
# Cela permet de voir si une transaction a eu un impact inhabituel
history_features['balance_variation'] = (
    history_df['current_account_balance'] - history_df['prev_account_balance']
)

# 5. [Optionnel] Encodage du statut s'il est présent (FAILED, SUCCESS, etc.)
# if 'status' in history_df.columns:
#     history_features['status_failed'] = (history_df['status'] == 'FAILED').astype(int)

# 6. [Optionnel] Encodage des frais si la colonne existe
# if 'transaction_fees' in history_df.columns:
#     history_features['transaction_fees_anomalous'] = history_df['transaction_fees'].apply(lambda x: 1 if x == 0 else 0)

# Afficher les premières lignes pour vérification
print("Aperçu des features extraites de history_cleaned.csv :")
print(history_features.head())

Aperçu des features extraites de history_cleaned.csv :
    user_id  hour  day_of_week  is_weekend  balance_variation
0  0.000735     0            3           0          -0.000141
1  0.000735     0            3           0          -0.000141
2  0.000735     0            3           0          -0.000141
3  0.000735     0            3           0          -0.000141
4  0.000735     0            3           0          -0.000141


In [4]:
# b. classe user

# Vérifier que les colonnes de date sont bien au format datetime
user_df['date_start'] = pd.to_datetime(user_df['date_start'])

# Créer un DataFrame pour stocker uniquement les features
user_features = pd.DataFrame()

# Clé d'identification principale
user_features['user_id'] = user_df['id']  # id de l'utilisateur (clé primaire)

# 1. Ancienneté du compte (en jours)
user_features['account_age_days'] = (pd.Timestamp.now() - user_df['date_start']).dt.days

# 2. Vérification du compte (1 = vérifié, 0 = non vérifié)
# Utiliser la bonne colonne selon le nom trouvé précédemment
user_features['is_verified_user'] = user_df['verif_user_1']

# 3. Variation de localisation (ville actuelle vs dernière connue)
user_features['location_variation'] = (user_df['host_city'] != user_df['last_host_location']).astype(int)

# 4. Incohérence géographique (même logique que location_variation)
user_features['host_location_incoherent'] = (user_df['host_city'] != user_df['last_host_location']).astype(int)

# 5. [Optionnel] Score utilisateur si disponible
if 'score' in user_df.columns:
    user_features['user_score'] = user_df['score']

# 6. [Optionnel] Nombre de transactions (utile pour identifier les profils dormants)
if 'nb_trans' in user_df.columns:
    user_features['transaction_count'] = user_df['nb_trans']

# Affichage pour vérification
print("Aperçu des features extraites de user_cleaned.csv :")
print(user_features.head())

Aperçu des features extraites de user_cleaned.csv :
    user_id  account_age_days  is_verified_user  location_variation  \
0  0.000000             20198               0.0                   1   
1  0.001827             20198               0.0                   1   
2  0.002558             20198               0.0                   1   
3  0.004750             20198               0.0                   1   
4  0.005115             20198               1.0                   1   

   host_location_incoherent  transaction_count  
0                         1           0.000000  
1                         1           0.000340  
2                         1           0.003574  
3                         1           0.139113  
4                         1           0.024760  


In [5]:
# c. classe proceed

# Création du DataFrame des features
proceed_features = pd.DataFrame()

# Clé de liaison pour les futures fusions
proceed_features['user_id'] = proceed_df['user_id']

# 1. Montant traité par la transaction (utile pour évaluer les anomalies de volume)
proceed_features['proceed_amount'] = proceed_df['amount']

# 2. Devise utilisée 
#. Encodage des devises déjà présent (on copie directement les colonnes)
currency_columns = [col for col in proceed_df.columns if col.startswith("currency_")]
proceed_features = pd.concat([proceed_features, proceed_df[currency_columns]], axis=1)

# 3. Indicateur : le wallet utilisé est-il connu ou non (NULL = suspect)
proceed_features['wallet_known_user'] = proceed_df['wallet'].notnull().astype(int)

# 4. [Optionnel] Service utilisé si colonne présente
if 'service_id' in proceed_df.columns:
    proceed_features['service_id'] = proceed_df['service_id']

# 5. [Optionnel] Type d’opération si présent
if 'operation_id' in proceed_df.columns:
    proceed_features['operation_id'] = proceed_df['operation_id']

# Affichage d’un aperçu des features extraites
print("Aperçu des features extraites de proceed_cleaned.csv :")
print(proceed_features.head())

Aperçu des features extraites de proceed_cleaned.csv :
   user_id  proceed_amount  currency_CDF  currency_DOGE  currency_LTC  \
0      0.0    2.000000e-16           0.0            0.0           0.0   
1      0.0    2.000000e-14           0.0            0.0           0.0   
2      0.0    5.000000e-15           0.0            0.0           0.0   
3      0.0    5.000000e-15           0.0            0.0           0.0   
4      0.0    5.000000e-15           0.0            0.0           0.0   

   currency_USD  currency_XAF  currency_XOF  wallet_known_user  service_id  
0           1.0           0.0           0.0                  1    0.035714  
1           0.0           1.0           0.0                  1    0.000000  
2           0.0           1.0           0.0                  1    0.000000  
3           0.0           1.0           0.0                  1    0.000000  
4           0.0           1.0           0.0                  1    0.000000  


In [6]:
# d. classe splogin_history


# === Étape 1 : Création du DataFrame des features ===
splogin_features = pd.DataFrame()

# Clé pour relier avec les autres sources
splogin_features["user_id"] = splogin_history_df["user_id"]

# 1. Nombre total de tentatives de connexion par utilisateur
login_counts = splogin_history_df.groupby("user_id").size()
splogin_features["total_logins"] = splogin_features["user_id"].map(login_counts)

# 2. Nombre de connexions réussies (status_SUCCESS = 1)
splogin_features["login_successes"] = splogin_history_df.groupby("user_id")["status_SUCCESS"].transform("sum")

# 3. Nombre d'échecs = total - succès
splogin_features["login_failures"] = splogin_features["total_logins"] - splogin_features["login_successes"]

# 4. Ratio de connexions échouées
splogin_features["failure_ratio"] = (
    splogin_features["login_failures"] / splogin_features["total_logins"]
).fillna(0)

# 5. Nombre de pays différents utilisés pour se connecter
country_counts = splogin_history_df.groupby("user_id")["country"].nunique()
splogin_features["unique_countries"] = splogin_features["user_id"].map(country_counts)

# 6. Nombre de villes différentes utilisées
city_counts = splogin_history_df.groupby("user_id")["city"].nunique()
splogin_features["unique_cities"] = splogin_features["user_id"].map(city_counts)

# === Étape 4 : Aperçu final ===
print("Aperçu des features extraites de splogin_cleaned.csv :")
print(splogin_features.head())

Aperçu des features extraites de splogin_cleaned.csv :
    user_id  total_logins  login_successes  login_failures  failure_ratio  \
0  0.005952            35              2.0            33.0       0.942857   
1  0.005952            35              2.0            33.0       0.942857   
2  0.006410           145              0.0           145.0       1.000000   
3  0.006410           145              0.0           145.0       1.000000   
4  0.006410           145              0.0           145.0       1.000000   

   unique_countries  unique_cities  
0                 2              7  
1                 2              7  
2                 3             10  
3                 3             10  
4                 3             10  


In [7]:
# e. classe bill
# Objectif : connaître l’activité de paiement de chaque utilisateur

# 1. Convertir la colonne 'proceed_at' en format datetime
# Si certaines dates sont mal formatées, elles deviendront NaT (Not a Time)
bill_df['proceed_at'] = pd.to_datetime(bill_df['proceed_at'], errors='coerce')

# 2. Supprimer les lignes où la date est invalide
bill_df = bill_df.dropna(subset=['proceed_at'])

# 3. Extraire uniquement la date (sans l’heure) pour grouper par jour
bill_df['jour'] = bill_df['proceed_at'].dt.date

# 4. Grouper par utilisateur et jour pour obtenir :
#    - le nombre de paiements effectués dans la journée
#    - le montant total payé cette journée
bill_features = bill_df.groupby(['user_id', 'jour']).agg(
    nb_paiements_par_jour=('amount', 'count'),
    montant_total_par_jour=('amount', 'sum')
).reset_index()

# 9. Afficher ou sauvegarder le résultat
print(bill_features.head())



   user_id        jour  nb_paiements_par_jour  montant_total_par_jour
0  0.00000  2021-11-18                      4                3.999999
1  0.00294  2021-07-14                      3                2.999999
2  0.00294  2021-07-18                      1                1.000000
3  0.00294  2021-07-22                      1                1.000000
4  0.00294  2021-07-24                      1                1.000000


In [8]:
# f.classe changes

# === Vérification si 'proceed_at' est bien une date ===
if 'proceed_at' in changes_df.columns:
    changes_df["proceed_at"] = pd.to_datetime(changes_df["proceed_at"], errors='coerce')

# === Création du DataFrame des features ===
changes_features = pd.DataFrame()

# 1. Identifiant utilisateur
if "user_id" in changes_df.columns:
    changes_features["user_id"] = changes_df["user_id"]
else:
    print("Attention : colonne 'user_id' absente du fichier changes_cleaned.csv")

# 2. Montant total reçu (in_amount)
if "in_amount" in changes_df.columns:
    changes_features["total_in_amount"] = changes_df.groupby("user_id")["in_amount"].transform("sum")

# 3. Montant total envoyé (out_amount)
if "out_amount" in changes_df.columns:
    changes_features["total_out_amount"] = changes_df.groupby("user_id")["out_amount"].transform("sum")

# 4. Nombre total de transactions effectuées
changes_features["nb_changes_transactions"] = changes_df.groupby("user_id")["id"].transform("count")

# 5. Moyenne des montants envoyés et reçus
changes_features["avg_in_amount"] = changes_df.groupby("user_id")["in_amount"].transform("mean")
changes_features["avg_out_amount"] = changes_df.groupby("user_id")["out_amount"].transform("mean")

# 6. Ratio in/out (attention aux divisions par zéro)
changes_features["in_out_ratio"] = (
    changes_features["total_in_amount"] / changes_features["total_out_amount"]
).replace([float("inf"), -float("inf")], 0).fillna(0)

# 7. Nombre de statuts SUCCESS (si one-hot encoding est déjà appliqué)
if "status_SUCCESS" in changes_df.columns:
    changes_features["nb_success_changes"] = changes_df.groupby("user_id")["status_SUCCESS"].transform("sum")
    
    # Ratio de succès
    changes_features["success_rate_changes"] = (
        changes_features["nb_success_changes"] / changes_features["nb_changes_transactions"]
    ).fillna(0)

# Supprimer les doublons dus au groupby
changes_features = changes_features.drop_duplicates(subset=["user_id"])

# === Aperçu final ===
print("Aperçu des caractéristiques extraites de changes_cleaned.csv :")
print(changes_features.head())

Aperçu des caractéristiques extraites de changes_cleaned.csv :
     user_id  total_in_amount  total_out_amount  nb_changes_transactions  \
0   0.000369         0.000003      1.753777e-06                        8   
1   0.000738         0.000051      4.090962e-05                        6   
9   0.000000         0.000136      2.396141e-04                       26   
13  0.152768         0.000008      1.368489e-08                        1   
19  0.148339         0.000002      9.756839e-06                        9   

    avg_in_amount  avg_out_amount  in_out_ratio  nb_success_changes  \
0    3.378889e-07    2.192221e-07      1.541308                 5.0   
1    8.450967e-06    6.818270e-06      1.239459                 6.0   
9    5.232670e-06    9.215927e-06      0.567786                24.0   
13   7.613354e-06    1.368489e-08    556.332959                 1.0   
19   2.468652e-07    1.084093e-06      0.227716                 2.0   

    success_rate_changes  
0               0.625000  

In [9]:
# f. classe account

# === Création du DataFrame des features ===
account_features = pd.DataFrame()

# 1. Clé utilisateur
if "user_id" in account_df.columns:
    account_features["user_id"] = account_df["user_id"]
else:
    print("Attention : colonne 'user_id' absente dans account_cleaned.csv")

# 2. Solde total des comptes par utilisateur
if "balance" in account_df.columns:
    account_features["total_balance"] = account_df.groupby("user_id")["balance"].transform("sum")

# 3. Solde moyen des comptes
account_features["avg_balance"] = account_df.groupby("user_id")["balance"].transform("mean")

# 4. Nombre de comptes possédés
account_features["nb_accounts"] = account_df.groupby("user_id")["wallet"].transform("count")

# 5. Somme du solde en attente (pending_balance)
if "pending_balance" in account_df.columns:
    account_features["total_pending_balance"] = account_df.groupby("user_id")["pending_balance"].transform("sum")

# 6. Somme du trade_balance si disponible
if "trade_balance" in account_df.columns:
    account_features["total_trade_balance"] = account_df.groupby("user_id")["trade_balance"].transform("sum")

# 7. Dernière mise à jour (peut être utile pour détecter des comptes dormants)
if "updated_at" in account_df.columns:
    account_df["updated_at"] = pd.to_datetime(account_df["updated_at"], errors="coerce")
    last_update = account_df.groupby("user_id")["updated_at"].transform("max")
    account_features["days_since_last_update"] = (pd.Timestamp.now() - last_update).dt.days

# Supprimer les doublons dus aux .transform
account_features = account_features.drop_duplicates(subset=["user_id"])

# === Aperçu final ===
print("Aperçu des caractéristiques extraites de account_cleaned.csv :")
print(account_features.head())

Aperçu des caractéristiques extraites de account_cleaned.csv :
     user_id  total_balance  avg_balance  nb_accounts  total_trade_balance  \
0   0.000732       0.754910     0.188727            4             2.816901   
4   0.002928       1.554216     0.194277            8             5.646313   
9   0.003294       1.511367     0.188921            8             5.633806   
13  0.003660       1.554380     0.194298            8             5.636632   
17  0.004026       0.754910     0.188727            4             2.816901   

    days_since_last_update  
0                    20198  
4                    20198  
9                    20198  
13                   20198  
17                   20198  


In [10]:
#fusion des features

# Renommer la colonne 'id' en 'user_id' pour permettre la fusion cohérente avec les autres fichiers
user_df = user_df.rename(columns={"id": "user_id"})

# Fonction utilitaire pour éviter les colonnes dupliquées
def safe_merge(df1, df2, on="user_id"):
    # Identifier les colonnes communes hors clé de jointure
    common_cols = [col for col in df1.columns if col in df2.columns and col != on]
    # Supprimer les colonnes dupliquées du second DataFrame
    df2_cleaned = df2.drop(columns=common_cols)
    # Effectuer la fusion (jointure gauche)
    return df1.merge(df2_cleaned, on=on, how="left")

# Initialiser le DataFrame final avec les données utilisateurs
merged_features = user_df.copy()

# Fusion successive avec toutes les autres tables nettoyées
merged_features = safe_merge(merged_features, history_df)
merged_features = safe_merge(merged_features, proceed_df)
merged_features = safe_merge(merged_features, splogin_history_df)
merged_features = safe_merge(merged_features, bill_df)
merged_features = safe_merge(merged_features, changes_df)
merged_features = safe_merge(merged_features, account_df)


In [11]:
# 5) NETTOYAGE FINAL

# Nettoyage final

# 1. Remplacer les NaN uniquement dans les colonnes numériques par 0
numeric_cols = merged_features.select_dtypes(include=['float64', 'int64']).columns
merged_features[numeric_cols] = merged_features[numeric_cols].fillna(0)

# 2. Vérifier les types de colonnes après remplacement
print(merged_features.dtypes)

# 3. Afficher un aperçu des premières lignes
merged_features.head()

user_id                    float64
username                   float64
date_start          datetime64[ns]
last_operation             float64
balance                    float64
                         ...      
currency_id_7.0            float64
currency_id_8.0            float64
currency_id_14.0           float64
currency_id_15.0           float64
currency_id_21.0           float64
Length: 90, dtype: object


,user_id,username,date_start,last_operation,balance,nb_trans,contact,rewards_balance,matricule,last_login,...,trade_balance,currency_id_2.0,currency_id_3.0,currency_id_4.0,currency_id_5.0,currency_id_7.0,currency_id_8.0,currency_id_14.0,currency_id_15.0,currency_id_21.0
0,0.0,0.001494,1970-01-01,0.855809,0.199932,0.0,0.000374,1.0,0.000374,0.000374,...,0.704225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.001494,1970-01-01,0.855809,0.199932,0.0,0.000374,1.0,0.000374,0.000374,...,0.704225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.001494,1970-01-01,0.855809,0.199932,0.0,0.000374,1.0,0.000374,0.000374,...,0.704225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.001494,1970-01-01,0.855809,0.199932,0.0,0.000374,1.0,0.000374,0.000374,...,0.704225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.001494,1970-01-01,0.855809,0.199932,0.0,0.000374,1.0,0.000374,0.000374,...,0.704225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Définir le chemin vers le dossier "features"
merged_path = os.path.join("..", "features")

# 2. Définir le nom complet du fichier
merged_file = os.path.join(merged_path, "features_transactions.csv")

# 3. Sauvegarder le fichier CSV
merged_features.to_csv(merged_file, index=False)

print(f"Features sauvegardées avec succès dans : {merged_file}")

Features sauvegardées avec succès dans : ..\features\features_transactions.csv
